In [1]:
import os

In [9]:
with open('RECORDS', 'r') as f:
    dirs = f.readlines()
dirs = [d.strip() for d in dirs]

In [12]:
import shutil

for dir in dirs:
    for file in os.listdir(dir):
        shutil.copy(os.path.join(dir, file), 'trainingchapman')

In [15]:
list = os.listdir('trainingchapman')

In [16]:
heads = []
for file in list:
    head = file.split('.')[0]
    heads.append(head)

In [19]:
heads = set(heads)

In [20]:
len(heads)

45153

In [23]:
with open('training2017/RECORDTrain.txt', 'r') as f:
    train = f.readlines()
train = [t.strip() for t in train]

In [26]:
with open('training2017/RECORDValid.txt', 'r') as f:
    val = f.readlines()
val = [v.strip() for v in val]

In [27]:
with open('training2017/RECORDTest.txt', 'r') as f:
    test = f.readlines()
test = [t.strip() for t in test]

In [28]:
len(train), len(val), len(test)

(7928, 300, 300)

In [ ]:
heads = sorted(heads)

In [ ]:
from sklearn.model_selection import train_test_split

In [56]:
heads.remove('RECORDS')

In [59]:
with open('trainingchapman/RECORDTotal.txt', 'w') as f:
    for head in heads:
        f.write(head + '\n')

In [ ]:
from wfdb import rdrecord

In [322]:
label_dict = {}
for head in heads:
    try:
        data = rdrecord(os.path.join('trainingchapman', head))
        labels = data.comments[2]
        labels = labels.split(':')[-1]
        labels = labels.split(',')
        label_dict[head] = labels
    except:
        print(head)
        break

In [323]:
len(label_dict)

45152

In [195]:
import pandas as pd

In [310]:
id2label = pd.read_csv('trainingchapman/ConditionNames_SNOMED-CT.csv', index_col=0)

In [311]:
id2label.drop('Full Name', axis=1, inplace=True)

In [312]:
id2label

,Snomed_CT
Acronym Name,
1AVB,270492004
2AVB,195042002
2AVB1,54016002
2AVB2,28189009
3AVB,27885002
...,...
SVT,426761007
AT,713422000
AVNRT,233896004


In [313]:
id_dict = id2label.to_dict()['Snomed_CT']

In [377]:
merge_dict = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'ST': 'GSVT',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'WAVN': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

In [359]:
id_dict

{'1AVB': 270492004,
 '2AVB': 195042002,
 '2AVB1': 54016002,
 '2AVB2': 28189009,
 '3AVB': 27885002,
 'ABI': 251173003,
 'ALS': 39732003,
 'APB': 284470004,
 'AQW': 164917005,
 'ARS': 47665007,
 'AVB': 233917008,
 'CCR': 251199005,
 'CR': 251198002,
 'ERV': 428417006,
 'FQRS': 164942001,
 'IDC': 698252002,
 'IVB': 698252002,
 'JEB': 426995002,
 'JPT': 251164006,
 'LBBB': 164909002,
 'LBBBB': 164909002,
 'LFBBB': 164909002,
 'LVH': 164873001,
 'LVQRSAL': 251146004,
 'LVQRSCL': 251148003,
 'LVQRSLL': 251147008,
 'MI': 164865005,
 'MIBW': 164865005,
 'MIFW': 164865005,
 'MILW': 164865005,
 'MISW': 164865005,
 'PRIE': 164947007,
 'PWC': 164912004,
 'QTIE': 111975006,
 'RAH': 446358003,
 'RBBB': 59118001,
 'RVH': 89792004,
 'STDD': 429622005,
 'STE': 164930006,
 'STTC': 428750005,
 'STTU': 164931005,
 'TWC': 164934002,
 'TWO': 59931005,
 'UW': 164937009,
 'VB': 11157007,
 'VEB': 75532003,
 'VFW': 13640000,
 'VPB': 17338001,
 'VPE': 195060002,
 'VET': 251180001,
 'WAVN': 195101003,
 'WPW': 743

In [315]:
len(id_dict)

63

In [379]:
code_dict = {}
for k, v in id_dict.items():
    if k in merge_dict:
        code_dict[v] = merge_dict[k]

In [380]:
code_dict

{195101003: 'GSVT',
 426177001: 'SB',
 426783006: 'SR',
 164889003: 'AFIB',
 427084000: 'GSVT',
 164890007: 'AFIB',
 427393009: 'SR',
 426761007: 'GSVT',
 713422000: 'GSVT',
 233896004: 'GSVT',
 233897008: 'GSVT'}

In [400]:
new_label_dict = {}
append_label_dict = {}
for k, vs in label_dict.items():
    new_label_dict[k] = []
    append_label_dict[k] = []
    for v in vs:
        v = int(v.strip())
        if v in code_dict:
            new_label_dict[k].append(code_dict[v])
            append_label_dict[k].append(code_dict[v])
        elif v in id_dict.values():
            append_label_dict[k].append(v)

In [401]:
cnt_mul = 0
cnt_non = 0
mul_list = []
non_list = []
for k, v in new_label_dict.items():
    if len(v) > 1:
        cnt_mul += 1
        mul_list.append(k)
    elif len(v) == 0:
        cnt_non += 1
        non_list.append(k)
print(cnt_mul, cnt_non)

1159 949


In [402]:
drop_list = non_list + mul_list

In [403]:
drop_list = sorted(drop_list)

In [212]:
df = pd.DataFrame.from_dict(label_dict, orient='index').dropna(axis=0, how='all')

In [386]:
len(heads)

45152

In [404]:
trains, tests = train_test_split(heads, train_size=0.6, test_size=0.4, shuffle=True, random_state=42)
valids, tests = train_test_split(tests, train_size=0.5, test_size=0.5, shuffle=True, random_state=42)

In [405]:
trains = sorted(trains)
valids = sorted(valids)
tests = sorted(tests)
len(trains), len(valids), len(tests)

(27091, 9030, 9031)

In [406]:
with open('trainingchapman/RECORDTotal.txt', 'w') as f:
    for head in heads:
        f.write(head + '\n')
with open('trainingchapman/RECORDTrain.txt', 'w') as f:
    for train in trains:
        f.write(train + '\n')
with open('trainingchapman/RECORDValid.txt', 'w') as f:
    for valid in valids:
        f.write(valid + '\n')
with open('trainingchapman/RECORDTest.txt', 'w') as f:
    for test in tests:
        f.write(test + '\n')

In [409]:
label_dict2 = {k: v for k, v in new_label_dict.items() if k not in drop_list}

In [414]:
label = pd.DataFrame.from_dict(label_dict2, orient='index', columns=['label'])

In [415]:
label.index.name = 'head'

In [417]:
label.to_csv('trainingchapman/REFERENCE4.csv')

In [418]:
labels = pd.read_csv('trainingchapman/REFERENCE4.csv')

In [419]:
labels

,head,label
0,JS00001,AFIB
1,JS00002,SB
2,JS00004,SB
3,JS00005,AFIB
4,JS00006,SB
...,...,...
43039,JS45385,GSVT
43040,JS45435,GSVT
43041,JS45507,GSVT
43042,JS45543,GSVT
